# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 16 баллов. 4 за сдачу в срок и 12 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [1]:
import string

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_text = reference_text.lower()
    recognized_text = recognized_text.lower()

    translator = str.maketrans('', '', string.punctuation)
    reference_text = reference_text.translate(translator)
    recognized_text = recognized_text.translate(translator)

    reference_words = reference_text.split()
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                cost = 0
            else:
                cost = 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,
                distance_matrix[i][j - 1] + 1,
                distance_matrix[i - 1][j - 1] + cost
            )

    # Расчет WER  (в процентах)
    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100
    return wer

wer = calculate_wer('труд мир балалайка', 'привет, мир!')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 66.67%


In [2]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('кто-то', 'кто', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [3]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

In [4]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    # Перенесите сюда код из задания 1.a.
    #wer = 
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_text = reference_text.lower()
    recognized_text = recognized_text.lower()

    translator = str.maketrans('', '', string.punctuation)
    reference_text = reference_text.translate(translator)
    recognized_text = recognized_text.translate(translator)

    reference_words = reference_text.split()
    recognized_words = recognized_text.split()

    # расстояние Левенштейна 
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]

    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i - 1] == recognized_words[j - 1]:
                cost = 0
            else:
                cost = 1
            distance_matrix[i][j] = min(
                distance_matrix[i - 1][j] + 1,
                distance_matrix[i][j - 1] + 1,
                distance_matrix[i - 1][j - 1] + cost
            )

    # Расчет WER  (в процентах)
    wer = (distance_matrix[len(reference_words)][len(recognized_words)] / len(reference_words)) * 100

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация
    i = len(reference_words)
    j = len(recognized_words)
    reference_ali = []
    hypotheses_ali =[]
    annotation = []
    correct = 0
    substitution = 0
    deletion = 0
    insertion = 0

    while i > 0 or j > 0:
        # Оба слова присутствуют
        if i > 0 and j > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j - 1] and reference_words[i - 1] == recognized_words[j - 1]:
            reference_ali.append(reference_words[i - 1])
            hypotheses_ali.append(recognized_words[j - 1])
            annotation.append("C")
            correct += 1
            i -= 1
            j -= 1
        # Замена
        elif i > 0 and j > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j - 1] + 1:
            reference_ali.append(reference_words[i - 1])
            hypotheses_ali.append(recognized_words[j - 1])
            annotation.append("S")
            substitution += 1
            i -= 1
            j -= 1
        # Вставка
        elif j > 0 and distance_matrix[i][j] == distance_matrix[i][j - 1] + 1:
            reference_ali.append("***")
            hypotheses_ali.append(recognized_words[j - 1])
            annotation.append("I")
            insertion += 1
            j -= 1
        # Удаление
        elif i > 0 and distance_matrix[i][j] == distance_matrix[i - 1][j] + 1:
            reference_ali.append(reference_words[i - 1])
            hypotheses_ali.append("***")
            annotation.append("D")
            deletion += 1
            i -= 1

    reference_ali.reverse()
    hypotheses_ali.reverse()
    annotation.reverse()
    ali = [reference_ali, hypotheses_ali, annotation]
    
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [5]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed
